# Modified 3D PMRNN

## Module Imports

In [1]:
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '1'
import tensorflow as tf
tf.compat.v1.disable_eager_execution()
config = tf.compat.v1.ConfigProto()
config.gpu_options.allow_growth = True
sess = tf.compat.v1.Session(config=config)

from tensorflow.keras import layers,losses #,Sequential,metrics
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import * 
# from tensorflow.keras.layers import Layer
# from tensorflow.keras import optimizers 
# from tensorflow.keras.optimizers.experimental import SGD
# from tensorflow.image import ssim

# import cv2
import numpy as np
import matplotlib.pyplot as plt
import sklearn
from mealpy.swarm_based import GWO
from sklearn.preprocessing import LabelEncoder
import time
import pickle
from datetime import datetime
import mealpy

from CONFIGURATION import CONFIGURATION
from ViTReconstruction import Reconstruction

FileNotFoundError: [WinError 2] The system cannot find the file specified: 'C:\\Users\\TRETEC\\AppData\\Roaming\\Python\\Python39\\site-packages\\matplotlib.libs'

In [2]:
# ignorable
print("TensorFlow version:", tf.__version__)
print("cv2 version:",cv2.__version__)
print("np version:",np.__version__)
print("matplotlib version:",matplotlib.__version__)
print("sklearn version:",sklearn.__version__)
print("mealpy version:",mealpy.__version__)

TensorFlow version: 2.5.0
cv2 version: 4.5.3
np version: 1.19.5


## Load new Data

### reading data

In [13]:
import Utils
image_data = Utils.read_data("Berea_2d25um_binary.raw")
image_data.shape

(1000, 1000, 1000)

### Voxel extraction 

In [14]:
voxels = Utils.extract_subvolumes(image_data)
voxels.shape

(64, 250, 256, 256, 1)

In [15]:
del image_data

### Train and test data extraction

In [16]:
X_train = voxels[:-6]
X_test = voxels[-6:]

In [17]:
del voxels

In [18]:
len(X_train)

58

### Save Traited data for uses

In [19]:
with open('X_train.npy', 'wb') as f:
    np.save(f, X_train)

In [20]:
with open('X_test.npy', 'wb') as f:
    np.save(f, X_test)

## TensorFlow GPU setup

In [21]:
#from tensorflow.python.compiler.tensorrt import trt_convert as trt
device = tf.config.list_physical_devices("GPU")
print(device)  # Check if GPU devices are visible
print(tf.test.is_built_with_cuda())  # Check if TensorFlow is built with CUDA support
#print(trt.trt_utils._pywrap_py_utils.get_linked_tensorrt_version())
tf.keras.backend.clear_session()

[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]
True


## Load Traited data for training (voxels)

In [24]:
X_train=np.load('X_train.npy')
X_train.shape

(58, 250, 256, 256, 1)

In [25]:
# for test only
X_train=X_train[:1]
with open('X_train_test.npy', 'wb') as f:
    np.save(f, X_train)
X_train.shape

(1, 250, 256, 256, 1)

In [26]:
X_train=np.load('X_train_test.npy')
X_train.shape

(1, 250, 256, 256, 1)

### Creating model input

In [27]:
learnedVoxels = []
inferenceVoxels = []
for voxel in X_train:
    learnedVoxels.append(voxel[:-1])
    inferenceVoxels.append(voxel[1:])

# learnedVoxels = np.array(learnedVoxels)
# inferenceVoxels = np.array(inferenceVoxels)

## Training 

### Lables

In [15]:
class Timer():
    def __init__(self,title=" ",p={}) -> None:
        self.title=title
        self.started=time.time()
        self.params=p
        print(f"{self.title} Started at : {datetime.fromtimestamp(self.started)}")
        self.ended=None
    def End(self):
        self.ended=time.time()
        print(f"{self.title} Ended at : {datetime.fromtimestamp(self.ended)} and it took {self.ended-self.started}s")
    def setParams(self,name,data):
        self.params[name]=data

    def toDuration(self):
        if self.ended == None:
            return -1
        return self.ended-self.started
    
    def __str__(self) -> str:
        return f"{self.title} Started at : {datetime.fromtimestamp(self.started)} | Ended at : {datetime.fromtimestamp(self.ended)} and it took {self.ended-self.started}s"

class TrainingData(Timer):
    def __init__(self,title=" ",p={}) -> None:
        super().__init__(title=title,p=p)
        self.samples=[]
    
    def SampleStarted(self,title=" ")->Timer:
        timer= Timer(title)
        self.samples.append(timer)
        return timer
    
    def __str__(self) -> str:
        return super().__str__()+" "+str(self.params)

In [37]:
OPT_ENCODER = LabelEncoder()
#OPT_ENCODER.fit(['SGD', 'RMSprop', 'Adagrad', 'Adadelta', 'Adam', 'Adamax', 'Nadam'])
OPT_ENCODER.fit(['SGD', 'RMSprop', 'Adagrad', 'Adam'])
trainings_duration=[]
trainings_epochs=[]

### Solution Decodation function

In [38]:
def decode_solution(solution):
    batch_size = 2**int(solution[0])
    
    learning_rate = solution[1]
    
    opt_int = int(solution[2])
    opt = OPT_ENCODER.inverse_transform([opt_int])[0]
    
    epoch = 10 *int(solution[3])
    
    num_filters = int(solution[4])
    
    latent_space_dim = 2**int(solution[5])
    
    reduced_dim = 2**int(solution[6])
    
    n_heads  = int(solution[7]) 
    hidden_size = 2** int(solution[8])
    enc_layers =  int(solution[9])
    
    return [batch_size,learning_rate,opt,epoch,num_filters,latent_space_dim,reduced_dim,n_heads,hidden_size,enc_layers]


### Objective Function

In [39]:

current_epoch=0

def objective_function(solution):
    tf.keras.backend.clear_session()
    global current_training
    print("\n###########################################################\n")
    print(f"\n--------------------------{current_iteration}/{current_training}--------------------------------\n")
    print("\n###########################################################\n")
    batch_size,learning_rate,opt,epoch,num_filters,latent_space_dim ,reduced_dim,n_heads,hidden_size,enc_layers = decode_solution(solution)
    reconstruction = Reconstruction(inputShape=CONFIGURATION["INPUT_SHAPE"],
                                    latent_space_dim=latent_space_dim,
                                    reducedDimension=reduced_dim,
                                    num_conv_layers=num_filters,
                                    learning_rate=learning_rate,
                                    batch_size=batch_size,
                                    epochs=epoch,
                                    opt=opt,
                                    )
    #reconstruction.summary()
    reconstruction.compile()
    histories = []
    params = {
        "inputShape": CONFIGURATION["INPUT_SHAPE"],
        "latent_space_dim": latent_space_dim,
        "reducedDimension": reduced_dim,
        "num_conv_layers": num_filters,
        "learning_rate": learning_rate,
        "batch_size": batch_size,
        "epochs": epoch,
        "opt": opt,
        "n_heads" : n_heads, 
        "hidden_size" :  hidden_size,
        "enc_layers" : enc_layers,
    }
    trainingTime = TrainingData(f"Epoch Training {current_training}", params)
    trainings_duration.append(trainingTime)
    training_loss=1

    for i in range(len(learnedVoxels)):
        print(f"---Currently working one voxel : {i +1}, voxels left : {len(learnedVoxels)-(i+1)} ")
        sampleTime = trainingTime.SampleStarted(f"Voxel Training {i+1}")
        try:
            # Code that may raise an OOM error
            training_result=reconstruction.train(learnedVoxels[i], inferenceVoxels[i])
            sampleTime.End()
            training_loss=training_result.history['loss'][-1]
            print(training_loss)
            if (training_loss > 0.01) or (i==1 and training_loss > 0.009)or (i>1 and training_loss > 0.008):
                trainingTime.setParams("Bad_params",True)
                break
        except MemoryError as e:
            # Code to handle the OOM error
            trainingTime.setParams("OMM_Errored",str(e))
            break
        except Exception as e:
            trainingTime.setParams("Error",str(e))
            break
        

    trainingTime.setParams("loss",training_loss)
    trainingTime.setParams("finished",True)
    trainingTime.End()

    current_epoch+=1
    
    """ reconstruction.save( save_folder=f"results/model_{batch_size}_{learning_rate}_{opt}_{epoch}_{num_filters}_{latent_space_dim}_{reduced_dim}") """
#     return min(histories, key=lambda x: float('inf') if x is None else x)
    return training_loss

### Configurations

In [40]:
LB = [3    , 0.001 , 0   , 2      ,  4  ,  6    , 7   , 3   , 6 ,  2 ,    6 ]
UB = [5.99 , 0.01  , 3.99, 4.99  ,  8.99,  8.99 , 9.99, 5.99, 7.99, 4.99, 8.99 ]


problem_dict = {
    "fit_func": objective_function,
    "lb": LB,
    "ub": UB,
    "minmax": "min",
    "verbose":True,
}
metaheuristic_configuration={
    "epoch":5,
    "pop_size":10
}
#save session for x epoch of the training
save_interval=1
session_file = 'gwo_session.pkl'


### Lunching the metaheuristic

In [ ]:
# Check if a session file exists
if os.path.exists(session_file):
    # Load the session and resume from the saved state
    with open(session_file, 'rb') as f:
        session_data = pickle.load(f)
    model = session_data['model']
    best_solution = session_data['best_solution']
    current_iteration = session_data['current_iteration']
    print(best_solution)
    print("Resuming from iteration", current_iteration)
else:
    # Create a new GWO optimization session
    model = GWO.BaseGWO(problem_dict, epoch=metaheuristic_configuration["epoch"], pop_size=metaheuristic_configuration["pop_size"])
    best_solution = None
    current_iteration = 0
    trainings_epochs.append({"epoch" :current_iteration,"trainings" :trainings_duration,"best_solution":best_solution})
    trainings_duration=[]

In [43]:


# Run the optimization loop
while current_iteration < metaheuristic_configuration["epoch"]:
    # Perform GWO iteration
    best_position, best_fitness = model.solve()

    # Update the best solution
    if best_solution is None or best_fitness < best_solution["fit"]:
        best_solution = {
            "fit": best_fitness,
            "position": best_position
        }

    current_iteration += 1
    trainings_epochs.append({"epoch" :current_iteration,"trainings" :trainings_duration,"best_solution":best_solution})
    trainings_duration=[]

    # Increment the iteration count
    # Save the session periodically
    if current_iteration % save_interval == 0:
        session_data = {
            "model": model,
            "best_solution": best_solution,
            "current_iteration": current_iteration
        }
        with open(session_file, 'wb') as f:
            pickle.dump(session_data, f)

2023/06/18 11:22:27 PM, INFO, mealpy.swarm_based.GWO.BaseGWO: Solving single objective optimization problem.
2023/06/18 11:22:27 PM, INFO, mealpy.swarm_based.GWO.BaseGWO: Solving single objective optimization problem.
2023/06/18 11:22:27 PM, INFO, mealpy.swarm_based.GWO.BaseGWO: >Epoch: 1, Current best: 0.001098904280711252, Global best: 0.001098904280711252, Runtime: 0.01415 seconds
2023/06/18 11:22:27 PM, INFO, mealpy.swarm_based.GWO.BaseGWO: >Epoch: 1, Current best: 0.001098904280711252, Global best: 0.001098904280711252, Runtime: 0.01415 seconds
2023/06/18 11:22:27 PM, INFO, mealpy.swarm_based.GWO.BaseGWO: >Epoch: 2, Current best: 0.001098904280711252, Global best: 0.001098904280711252, Runtime: 0.01651 seconds
2023/06/18 11:22:27 PM, INFO, mealpy.swarm_based.GWO.BaseGWO: >Epoch: 2, Current best: 0.001098904280711252, Global best: 0.001098904280711252, Runtime: 0.01651 seconds
2023/06/18 11:22:27 PM, INFO, mealpy.swarm_based.GWO.BaseGWO: >Epoch: 3, Current best: 0.00109890428071125


###########################################################


--------------------------0--------------------------------


###########################################################

Epoch Training Started at : 2023-06-18 23:22:27.234473
currently working one voxel : 1, voxels left : 57 
Voxel Training 1 Started at : 2023-06-18 23:22:27.234473
Voxel Training 1 Ended at : 2023-06-18 23:22:27.234473 and it took 0.0s
currently working one voxel : 2, voxels left : 56 
Voxel Training 2 Started at : 2023-06-18 23:22:27.234473
Voxel Training 2 Ended at : 2023-06-18 23:22:27.234473 and it took 0.0s
currently working one voxel : 3, voxels left : 55 
Voxel Training 3 Started at : 2023-06-18 23:22:27.234473
Voxel Training 3 Ended at : 2023-06-18 23:22:27.234473 and it took 0.0s
currently working one voxel : 4, voxels left : 54 
Voxel Training 4 Started at : 2023-06-18 23:22:27.234473
Voxel Training 4 Ended at : 2023-06-18 23:22:27.234473 and it took 0.0s
currently working one voxel : 5, voxels

2023/06/18 11:22:27 PM, INFO, mealpy.swarm_based.GWO.BaseGWO: >Epoch: 4, Current best: 0.0013402225301088425, Global best: 0.0013402225301088425, Runtime: 0.01839 seconds
2023/06/18 11:22:27 PM, INFO, mealpy.swarm_based.GWO.BaseGWO: >Epoch: 5, Current best: 0.0013402225301088425, Global best: 0.0013402225301088425, Runtime: 0.01729 seconds
2023/06/18 11:22:27 PM, INFO, mealpy.swarm_based.GWO.BaseGWO: >Epoch: 5, Current best: 0.0013402225301088425, Global best: 0.0013402225301088425, Runtime: 0.01729 seconds
2023/06/18 11:22:27 PM, INFO, mealpy.swarm_based.GWO.BaseGWO: >Epoch: 1, Current best: 0.0022976624055759645, Global best: 0.0022976624055759645, Runtime: 0.02384 seconds
2023/06/18 11:22:27 PM, INFO, mealpy.swarm_based.GWO.BaseGWO: >Epoch: 1, Current best: 0.0022976624055759645, Global best: 0.0022976624055759645, Runtime: 0.02384 seconds
2023/06/18 11:22:27 PM, INFO, mealpy.swarm_based.GWO.BaseGWO: >Epoch: 2, Current best: 0.0012409403380282194, Global best: 0.0012409403380282194,


###########################################################


--------------------------111--------------------------------


###########################################################

Epoch Training Started at : 2023-06-18 23:22:27.435518
currently working one voxel : 1, voxels left : 57 
Voxel Training 1 Started at : 2023-06-18 23:22:27.435518
Voxel Training 1 Ended at : 2023-06-18 23:22:27.435518 and it took 0.0s
currently working one voxel : 2, voxels left : 56 
Voxel Training 2 Started at : 2023-06-18 23:22:27.435518
Voxel Training 2 Ended at : 2023-06-18 23:22:27.435518 and it took 0.0s
currently working one voxel : 3, voxels left : 55 
Voxel Training 3 Started at : 2023-06-18 23:22:27.435518
Voxel Training 3 Ended at : 2023-06-18 23:22:27.435518 and it took 0.0s
currently working one voxel : 4, voxels left : 54 
Voxel Training 4 Started at : 2023-06-18 23:22:27.435518
Voxel Training 4 Ended at : 2023-06-18 23:22:27.435518 and it took 0.0s
currently working one voxel : 5, voxe

2023/06/18 11:22:27 PM, INFO, mealpy.swarm_based.GWO.BaseGWO: >Epoch: 2, Current best: 0.0027994813370126343, Global best: 0.0027994813370126343, Runtime: 0.10440 seconds
2023/06/18 11:22:27 PM, INFO, mealpy.swarm_based.GWO.BaseGWO: >Epoch: 2, Current best: 0.0027994813370126343, Global best: 0.0027994813370126343, Runtime: 0.10440 seconds
2023/06/18 11:22:27 PM, INFO, mealpy.swarm_based.GWO.BaseGWO: >Epoch: 3, Current best: 0.0027994813370126343, Global best: 0.0027994813370126343, Runtime: 0.01531 seconds
2023/06/18 11:22:27 PM, INFO, mealpy.swarm_based.GWO.BaseGWO: >Epoch: 3, Current best: 0.0027994813370126343, Global best: 0.0027994813370126343, Runtime: 0.01531 seconds
2023/06/18 11:22:27 PM, INFO, mealpy.swarm_based.GWO.BaseGWO: >Epoch: 4, Current best: 0.0012481812014126879, Global best: 0.0012481812014126879, Runtime: 0.01566 seconds
2023/06/18 11:22:27 PM, INFO, mealpy.swarm_based.GWO.BaseGWO: >Epoch: 4, Current best: 0.0012481812014126879, Global best: 0.0012481812014126879,

Voxel Training 38 Started at : 2023-06-18 23:22:27.617559
Voxel Training 38 Ended at : 2023-06-18 23:22:27.706579 and it took 0.0890200138092041s
currently working one voxel : 39, voxels left : 19 
Voxel Training 39 Started at : 2023-06-18 23:22:27.706579
Voxel Training 39 Ended at : 2023-06-18 23:22:27.706579 and it took 0.0s
currently working one voxel : 40, voxels left : 18 
Voxel Training 40 Started at : 2023-06-18 23:22:27.706579
Voxel Training 40 Ended at : 2023-06-18 23:22:27.706579 and it took 0.0s
currently working one voxel : 41, voxels left : 17 
Voxel Training 41 Started at : 2023-06-18 23:22:27.706579
Voxel Training 41 Ended at : 2023-06-18 23:22:27.706579 and it took 0.0s
currently working one voxel : 42, voxels left : 16 
Voxel Training 42 Started at : 2023-06-18 23:22:27.706579
Voxel Training 42 Ended at : 2023-06-18 23:22:27.706579 and it took 0.0s
currently working one voxel : 43, voxels left : 15 
Voxel Training 43 Started at : 2023-06-18 23:22:27.706579
Voxel Traini

In [45]:
# print("\n".join( [str(training) for epoch in trainings_epochs for training in epoch]))
print(f"Best solution : {best_solution['fit']}")
batch_size,learning_rate,opt,epoch,num_filters,latent_space_dim ,reduced_dim,n_heads,hidden_size,enc_layers = decode_solution(best_solution["position"])
print(f"Batch size : {batch_size}, learning_rate : {learning_rate}, opt : {opt}, epoch : {epoch}, num_filters : {num_filters}, latent_space_dim : {latent_space_dim}, reduced_dim : {reduced_dim}, n_heads : {n_heads}, hidden_size : {hidden_size}, enc_layers : {enc_layers}")

Best solution : 0.001098904280711252
Batch size : 32, learning_rate : 0.003505615363045197, opt : SGD, epoch : 40, num_filters : 8, latent_space_dim : 256, reduced_dim : 512, n_heads : 5, hidden_size : 128, enc_layers : 4


### Save History

In [ ]:
model.history.save_global_objectives_chart(filename="hello/goc")
model.history.save_local_objectives_chart(filename="hello/loc")

model.history.save_global_best_fitness_chart(filename="hello/gbfc")
model.history.save_local_best_fitness_chart(filename="hello/lbfc")

model.history.save_runtime_chart(filename="hello/rtc")

model.history.save_exploration_exploitation_chart(filename="hello/eec")

model.history.save_diversity_chart(filename="hello/dc")

model.history.save_trajectory_chart(list_agent_idx=[3, 5], selected_dimensions=[3], filename="hello/tc")

### Remove old sessions

In [42]:
if os.path.exists(session_file):
    os.remove(session_file)

In [47]:
for training in trainings_epochs:
    print(len(training["trainings"]))

0
1
60
60
60
60
60


In [28]:
import json
jd=[]
for training in trainings_epochs:
    json_data=[]
    for obj in training["trainings"]:
        print(obj.__dict__)
        #json_data.append(json.dumps(obj.__dict__))
    jd.append(json_data)  

# Convert the object to JSON

# Print the JSON data
print(jd)

{'title': 'Epoch Training', 'started': 1687122277.0098681, 'params': {'inputShape': (256, 256, 1), 'latent_space_dim': 64, 'reducedDimension': 256, 'num_conv_layers': 6, 'learning_rate': 0.008622156700456095, 'batch_size': 32, 'epochs': 20, 'opt': 'SGD'}, 'ended': 1687122277.0099683, 'samples': [<__main__.Timer object at 0x7f58978fa7f0>]}
{'title': 'Epoch Training', 'started': 1687122277.0148833, 'params': {'inputShape': (256, 256, 1), 'latent_space_dim': 128, 'reducedDimension': 128, 'num_conv_layers': 6, 'learning_rate': 0.008670313294691801, 'batch_size': 8, 'epochs': 30, 'opt': 'Adagrad'}, 'ended': 1687122277.0149374, 'samples': [<__main__.Timer object at 0x7f5897575f40>]}
{'title': 'Epoch Training', 'started': 1687122277.0168173, 'params': {'inputShape': (256, 256, 1), 'latent_space_dim': 64, 'reducedDimension': 256, 'num_conv_layers': 7, 'learning_rate': 0.005740313120125359, 'batch_size': 16, 'epochs': 40, 'opt': 'RMSprop'}, 'ended': 1687122277.0168881, 'samples': [<__main__.Tim

In [19]:
histories = []
for i in range(len(learnedVoxels)):
    print(f"currently working one voxel : {i +1}, voxels left : {len(learnedVoxels)-(i+1)} ")
    histories.append(Reconstruction.train(learnedVoxels[i],inferenceVoxels[i]))

currently working one voxel : 1, voxels left : 57 


TypeError: train() missing 1 required positional argument: 'inputs2'